In [1]:
import json
from pprint import pprint
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas.io.json import json_normalize

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [19]:
import sys
sys.path.insert(0, '../utils')
import data_utils

sys.path.insert(0, '../models')
import log_reg_word_embeddings

In [23]:
df_train = data_utils.load_data_set('../data/train/train.csv')
df_train_x = df_train[['head.word', 'tail.word', 'sentence']]
df_train_y = df_train['relation']

In [15]:
model = log_reg_word_embeddings.LogisticRegression()

In [24]:
model.fit(df_train_x, df_train_y)

/home/tony/source/relation-extraction/venv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


ValueError: could not convert string to float: 'We begin in 1919 , as refugees from Odessa , fleeing from the Red Army , settle near Thessaloniki .'